<a href="https://colab.research.google.com/github/Cknowles11/DS-Unit-2-Linear-Models/blob/master/Copy_of_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [148]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# # If you're working locally:
# else:
#     DATA_PATH = '../data/'
    
# # Ignore this Numpy warning when using Plotly Express:
# # FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
# import warnings
# warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [149]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [150]:
df['created'] = pd.to_datetime(df['created'])

In [209]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,total_rooms,total_amenities,description_length,lat_long
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0,588.0,"40.7145,-73.9425"
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,5,8.0,"40.7947,-73.9667"
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,3,691.0,"40.7388,-74.0018"
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2,492.0,"40.7539,-73.9677"
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,1,479.0,"40.8241,-73.9493"


In [152]:
df['created']

0       2016-06-24 07:54:24
1       2016-06-12 12:19:27
2       2016-04-17 03:26:41
3       2016-04-18 02:22:02
4       2016-04-28 01:32:41
                ...        
49347   2016-06-02 05:41:05
49348   2016-04-04 18:22:34
49349   2016-04-16 02:13:40
49350   2016-04-08 02:13:33
49351   2016-04-12 02:48:07
Name: created, Length: 48817, dtype: datetime64[ns]

In [208]:
df['lat_long'] = df['latitude'].map(str) + "," + df['longitude'].map(str)

# Attempt at reverse geocoding lat and lon.

In [201]:
pip install geopy


In [216]:
import geopandas as gpd 
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
from tqdm import tqdm_pandas

In [229]:
locator = Nominatim(user_agent = 'myGeocoder', timeout = 20)
my_geocode = RateLimiter(locator.reverse, min_delay_seconds=.0000000001)

In [ ]:
# tqdm_pandas()
# It took too long to process all the information.
df['Address'] = df['lat_long'].apply(my_geocode)

# Feature Engineering

In [153]:
df['total_rooms'] = df['bathrooms'] + df['bedrooms']

In [154]:
df['total_rooms'].isnull().any()

False

In [155]:
df['total_amenities'] = df.iloc[:, -27:-3].sum(axis = 1)

In [156]:
df['total_amenities'].isnull().any()

False

In [157]:
df['description_length'] = df['description'].str.len()

In [158]:
df['description_length'] = df['description_length'].fillna(0)

In [159]:
df['description_length'].isnull().any()

False

In [160]:
bool_series = pd.isnull(df['description_length'])
bool_series

0        False
1        False
2        False
3        False
4        False
         ...  
49347    False
49348    False
49349    False
49350    False
49351    False
Name: description_length, Length: 48817, dtype: bool

In [161]:
df[bool_series]

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,total_rooms,total_amenities,description_length


# Train/Test Split
Train

In [198]:
train = df[df['created'] < '2016-06-01']
test = df[df['created'] >= '2016-06']

In [163]:
print(train.shape)
test.shape

(31844, 37)


(16973, 37)

In [164]:
from sklearn.linear_model import LinearRegression

In [165]:
from sklearn.metrics import mean_absolute_error

In [166]:
my_model = LinearRegression()

In [167]:
features = ['total_amenities', 'total_rooms', 'description_length']
target = 'price'
X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

In [168]:
my_model.fit(X_train,y_train)
my_model.coef_

array([ 1.05615601e+02,  7.67218378e+02, -1.53655474e-02])

In [169]:
my_model.intercept_

997.4690246814548

In [170]:
y_pred = my_model.predict(X_train)
mae = mean_absolute_error(y_train,y_pred)

In [171]:
print(f'Train Error: {mae:.0f} dollars')

Train Error: 856 dollars


Test

In [172]:
y_pred = my_model.predict(X_test)
mae = mean_absolute_error(y_test,y_pred)
print(f'Test Error: {mae:.0f} dollars')

Test Error: 868 dollars


# Diff Features Test

In [199]:
features = ['total_rooms', 'elevator']
X_train = train[features]
X_test = test[features]

In [ ]:
my_model.fit(X_train, y_train)
print(my_model.coef_)
my_model.intercept_

In [189]:
y_pred = my_model.predict(X_train)

In [190]:
mae = mean_absolute_error(y_train,y_pred)
print(f'Train Error: {mae:.0f} dollars')

Train Error: 1147 dollars


In [185]:
y_pred = my_model.predict(X_test)

In [186]:
mae = mean_absolute_error(y_test,y_pred)
print(f'Test Error: {mae:.0f} dollars')

Test Error: 826 dollars
